# Hysteresis State models

In [1]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Model Equations

Herein, we outline the equations for the "Current Sigmoid" model, as described in Ai et al. (2022), the Differential Capacity Hysteresis State open-circuit potential model, as described in Wycisk (2022), and the Hysteresis State open-circuit potential model, as described in Axen (2022). In this notebook we collectively paraphrase all the original literature models into a common notation and definition. For the original model expressions please refer to the appropriate references.

In all of the models, the open-circuit potential is given by

$$U = \frac{1+h}{2} U_{\text{delith}} + \frac{1-h}{2} U_{\text{lith}},$$

where $h$ is a hysteresis state variable, $U_{\text{delith}}$ is the delithiation open-circuit potential, and $U_{\text{lith}}$ is the lithiation open-circuit potential. The hysteresis state variable $h$ is defined between -1 and 1, where 1 corresponds to the delithiation branch and -1 corresponds to the lithiation branch.

### Current Sigmoid (Ai)
This approach uses a sigmoid function to switch between the delithiation and lithiation branches of the open-circuit potential, depending on the sign of the current. The original paper gives the following expression for the open-circuit potential:

$$ U = \text{sigmoid}\left(-\frac{KI}{Q_{\text{cell}}}\right) U_{\text{delith}} + \text{sigmoid}\left(\frac{KI}{Q_{\text{cell}}}\right) U_{\text{lith}}
$$

Where $K$ is a fitting parameter (in the current implementation this is hard-coded to 100, as in the original publication), $I$ is the cell current, and $Q_{\text{cell}}$ is the cell capacity. To simplify the comparison with the other models, we can rewrite this expression in terms of the hysteresis state variable $h$ given by

$$h = 1 - 2 \, \text{sigmoid}\left(-\frac{KI}{Q_{\text{cell}}}\right).$$

### Hysteresis State Variable (Axen)
In this model, the state variable $h(z,t)$ is both stoichiometry and time-dependent, and is governed by the following ODE:
$$ \frac{dh(z,t)}{dt} = \gamma \left( \frac{i_{\text{vol}}}{F \, c_{\text{max}} \, \epsilon}\right) \frac{(1 - \text{sgn}(i_{\text{vol}}) \, h(z,t))}{2},$$
where $i_{\text{vol}}$ is the volumetric interfacial current density, $F$ is the Faraday constant, $c_{\text{max}}$ is the maximum lithium concentration in the particles and $\epsilon$ is the active material volume fraction. In this model, the rate parameter $\gamma$ is given by
$$\gamma = \left( \frac{1-\text{sgn}(i_{\text{vol}})}{2}  \gamma_{\text{lith}} + \frac{1+\text{sgn}(i_{\text{vol}})}{2} \gamma_{\text{delith}}\right),$$
which allows for a different decay rate during delithiation ($\gamma_{\text{delith}}$) and lithiation ($\gamma_{\text{lith}}$).
Note that this implementation is a reformulation of the governing equations in the original paper. Also note that $\gamma$ is dimensionless.


### Differential Capacity Hysteresis State (Wycisk)
As in the Axen model, the state variable $h(z,t)$ is both stoichiometry and time-dependent, and is governed by the following ODE:
$$ \frac{dh(z,t)}{dt} = \left(\frac{\gamma \cdot i_{\text{surf}}}{Q_{\text{cell}}}\right)\frac{1 - \text{sgn}(i_{\text{surf}}) \, h(z,t)}{2}, $$

where $ \gamma(z) $ is given by

$$ \gamma(z) = \Gamma(z) \cdot \frac{1}{\left(C_{\text{diff}}\left(z\right)\right)^{x}}.$$

Here $C_{\text{diff}}(z)$ is the differential capacity with respect to potential, expressed as 

$$ C_{\text{diff}}(z) = \frac{dQ}{dU_{\text{eq}}(z)},$$
where $Q$ is the capacity of the phase of active material experiencing the voltage hysteresis and $U_{\text{eq}}$ is the average open-circuit potential of the phase of active material experiencing the voltage hysteresis. The remaining parameters are $\Gamma$ and $x$ which are both fitting parameters that affect the response of the hysteresis state decay when passing charge in either direction.

Warning: in this implementation, the units of both $\gamma$ and $\Gamma$ are unusual. The parameter $\gamma$ has units $\text{m}^2$ and $\Gamma$ has units $\text{m}^2\text{V}^{x}\text{A}^{-x}\text{h}^{-x}$. Also note that the units of $Q$ are $\text{A}\text{h}$, but the time $t$ is expressed in seconds.


## Comparing the different hysteresis state models

We now compare the behavior of the different hysteresis state models. First we define the models, which are all based on the DFN model with a composite negative electrode comprised of two particles phases. One phase is modelled using a single open-circuit potential, while the other is modelled using a hysteresis state model. The positive electrode is modelled using a single open-circuit potential.

In [2]:
models = {
    "current sigmoid": pybamm.lithium_ion.DFN(
        {
            "open-circuit potential": (("single", "current sigmoid"), "single"),
            "particle phases": ("2", "1"),
            "thermal": "lumped",
        }
    ),
    "Wycisk": pybamm.lithium_ion.DFN(
        {
            "open-circuit potential": (("single", "Wycisk"), "single"),
            "particle phases": ("2", "1"),
            "thermal": "lumped",
        }
    ),
    "Axen": pybamm.lithium_ion.DFN(
        {
            "open-circuit potential": (("single", "Axen"), "single"),
            "particle phases": ("2", "1"),
            "thermal": "lumped",
        }
    ),
}

Next we define the parameter set, which is for a graphite and silicon negative electrode with a NMC positive electrode. We add some 

In [3]:
parameters = pybamm.ParameterValues("Chen2020_composite")
parameters.update(
    {
        "Negative current collector density [kg.m-3]": 8933.0,
        "Negative current collector specific heat capacity [J.kg-1.K-1]": 385.0,
        "Negative current collector thermal conductivity [W.m-1.K-1]": 398.0,
        "Negative electrode density [kg.m-3]": 1555.0,
        "Negative electrode specific heat capacity [J.kg-1.K-1]": 1437.0,
        "Negative electrode thermal conductivity [W.m-1.K-1]": 1.58,
        "Separator density [kg.m-3]": 1017.0,
        "Separator specific heat capacity [J.kg-1.K-1]": 1978.0,
        "Separator thermal conductivity [W.m-1.K-1]": 0.34,
        "Positive electrode density [kg.m-3]": 3699.0,
        "Positive electrode specific heat capacity [J.kg-1.K-1]": 1270.0,
        "Positive electrode thermal conductivity [W.m-1.K-1]": 1.04,
        "Positive current collector density [kg.m-3]": 2702.0,
        "Positive current collector specific heat capacity [J.kg-1.K-1]": 903.0,
        "Positive current collector thermal conductivity [W.m-1.K-1]": 238.0,
        "Total heat transfer coefficient [W.m-2.K-1]": 5.0,
    },
    check_already_exists=False,
)

Next, we need to add the additional parameters required by the model. In this example, $K_{\text{lith}} = K_{\text{delith}} = 100$ for the HS model, but it is recommended to test different values for the decay rate aiming for good agreement to experiment if the information is available, or a smooth transition between the lithiation and delithiation branches of the hysteresis. For example, depending on the case study, an acceptable value for $K_{\text{lith}}$ and $K_{\text{delith}}$ could be $10$ or lower.

In [4]:
# Wycisk parameters
parameters_wycisk = parameters.copy()

# get the lithiation and delithiation functions and define the average OCP
lithiation_ocp = parameters_wycisk["Secondary: Negative electrode lithiation OCP [V]"]
delithiation_ocp = parameters_wycisk[
    "Secondary: Negative electrode delithiation OCP [V]"
]


def ocp_avg(sto):
    return (lithiation_ocp(sto) + delithiation_ocp(sto)) / 2


parameters_wycisk.update(
    {
        "Secondary: Negative electrode OCP [V]": ocp_avg,
        "Secondary: Negative particle hysteresis decay rate": 100,
        "Secondary: Negative particle hysteresis switching factor": 10,
        "Secondary: Initial hysteresis state in negative electrode": 0,
    },
    check_already_exists=False,
)

# Axen parameters
parameters_axen = parameters.copy()
parameters_axen.update(
    {
        "Secondary: Negative particle lithiation hysteresis decay rate": 100,
        "Secondary: Negative particle delithiation hysteresis decay rate": 100,
        "Secondary: Initial hysteresis state in negative electrode": 0,
    },
    check_already_exists=False,
)

parameter_values = {
    "current sigmoid": parameters,
    "Wycisk": parameters_wycisk,
    "Axen": parameters_axen,
}

Then we define the experiment, which is a discharge and charge cycle at 1C.

In [5]:
experiment = pybamm.Experiment(
    [
        (
            "Discharge at 1C for 1 hour or until 2.5 V",
            "Rest for 15 minutes",
            "Charge at 1C until 4.2 V",
            "Hold at 4.2 V until 0.05 C",
            "Rest for 15 minutes",
        ),
    ]
)

We then loop over the models and solve them.

In [6]:
solutions = {}
for name, model in models.items():
    sim = pybamm.Simulation(
        model, experiment=experiment, parameter_values=parameter_values[name]
    )
    solutions[name] = sim.solve(calc_esoh=False)

/Users/robertwtimms/Documents/PyBaMM/src/pybamm/simulation.py:122: UserWarning: The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.
  self._solver = solver or self._model.default_solver


Finally, we plot the results.

In [7]:
output_variables = [
    "Current [A]",
    "Negative electrode secondary stoichiometry",
    "Voltage [V]",
    "Volume-averaged cell temperature [K]",
    "Total heating [W.m-3]",
    "Hysteresis electrochemical heating [W.m-3]",
    "X-averaged negative electrode secondary hysteresis state",
    "X-averaged negative electrode secondary open-circuit potential [V]",
]

pybamm.dynamic_plot(
    list(solutions.values()),
    labels=list(solutions.keys()),
    colors=["black", "blue", "red"],
    linestyles=["-", "--", "-."],
    output_variables=output_variables,
)

interactive(children=(FloatSlider(value=0.0, description='t', max=3.0035462852073542, step=0.03003546285207354…